## On the Theory of Policy Gradient Methods: Optimality, Approximation, and Distribution Shift


## イントロ
### 方策勾配法って理論的な収束とか研究しつくされているのか？
実際に理論系の研究は行われています。その多くが、収束性を滑らかさで仮定し、最適性を目的関数の停留点収束で示しているだけです。しかし、収束速度や方策を近似する関数の性質から発生する問題(直接パラメータ化とソフトマックスパラメータ化ではどのように近似誤差が発生するかなど)、有限の試行における方策勾配法の挙動といった疑問がまだ残っています。

### 本ページについて
主に直接パラメータ化とソフトマックスパラメータ化における理論の比較をおこないます。


### Preliminaly
* 割引無限マルコフ決定過程$M=(\mathcal{S}, \mathcal{A}, P, r, \gamma, \rho)$は、有限状態空間 $\mathcal{S}$；有限行動空間 $\mathcal{A}$；遷移カーネル$P$；報酬関数$r:\mathcal{S}\times\mathcal{A}\rightarrow[0,1]$；割引率 $\gamma \in [0,1)$；$\mathcal{S}$上の初期状態分布 $\rho$で構成されるタプルです。

**方策**
* 決定論的方策：$\pi:\mathcal{S}\rightarrow\mathcal{A}$
* 確率的方策：$\pi: \mathcal{S} \rightarrow \Delta(\mathcal{A})$ 

**価値関数**
* 状態価値関数
$$
V^{\pi}(s) := \mathbb{E} \left[ \sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) | \pi, s_0=s \right]
$$
あるいは、
$$
V^{\pi}(\rho) := \mathbb{E}_{s_0 \sim \rho} \left[ V^{\pi}(s_0) \right]
$$

* 行動価値関数
$$
Q^{\pi}(s, a) = \mathbb{E} \left[ \sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) | \pi, s_0=s, a_0=a \right]
$$

* アドバンテージ関数
$$
\quad A^{\pi}(s, a) := Q^{\pi}(s, a) - V^{\pi}(s)
$$

* Occupancy Measure
$$
\begin{equation*}
d^{\pi}_{s_0}(s) := (1-\gamma) \sum_{t=0}^{\infty} \gamma^t \operatorname{Pr}^{\pi}(s_t=s|s_0) \tag{4}
\end{equation*}
$$


**方策のパラメータ化**
* 直接パラメータ化
$$
\begin{equation*}
\pi_{\theta}(a \mid s) = \theta_{s, a} \tag{2}
\end{equation*}
$$
* ソフトマックスパラメータ化
$$
\begin{equation*}
\pi_{\theta}(a \mid s) = \frac{\exp(\theta_{s, a})}{\sum_{a' \in \mathcal{A}} \exp(\theta_{s, a'})} \tag{3}
\end{equation*}
$$



**非凹面性(補題3.1.)**
直接パラメータ化とソフトマックスパラメータ化は、$\max_{\theta \in \Theta} V^{\pi_{\theta}}(\rho)$という最適化問題において$V^{\pi_{\theta}}(s)$が非凹面であることがいえます。つまり凸最適化のツールが使えません。以下はその例です。
![](https://cdn.mathpix.com/cropped/2024_12_21_e87d8fe522bc7e580bf5g-11.jpg?height=432&width=522&top_left_y=250&top_left_x=346)
**TODO**説明追加

**Performance Lemma(補題3.2)**
すべての方策$\pi$, $\pi'$ および状態$s_0$について、
$$
V^{\pi}(s_0) - V^{\pi'}(s_0) = \frac{1}{1-\gamma} \mathbb{E}_{s \sim d^{\pi}_{s_0}} \mathbb{E}_{a \sim \pi(\cdot|s)} \left[ A^{\pi'}(s, a) \right]
$$
が成立する。

### 直接パラメータ化
直接パラメータ化は、基本的に制約が課されます。その制約の上で勾配上昇を行い、パラメータを更新します。更新時に、パラメータが制約を違反すれば制約内の点に射影をし直します。このような方法を**射影勾配法**といいます。

**勾配**
$$
\begin{equation*}
\frac{\partial V^{\pi}(\mu)}{\partial \pi(a|s)} = \frac{1}{1-\gamma} d^{\pi}_{\mu}(s) Q^{\pi}(s, a) \tag{7}
\end{equation*}
$$

つぎに求めたいのは、直接パラメータ化方策の勾配の停留点が大域的最適解であるということを示すことです(最適性)。非凹面性により、これを示すことができる凸性が明示的に言えませんが、**勾配支配**という概念をみたすことで、停留点が大域的最適解であるということが言えます。

**勾配支配(Gradient dominance)**
一般的には、ある関数$f(\theta)$があって、全ての$\theta \in \Theta$に対して、
$$
f(\theta^*) - f(\theta) = O(G(\theta))
$$
を満たす場合に勾配支配の特性を満たすといいます。ビッグオーで右辺が書かれていますが、最適パラメータと任意のパラメータの差分が何らかの値で抑えられるという認識でかまいません。


**補題4.1 (勾配支配)**

直接的な方策パラメータ化の場合、すべての状態分布 $\mu, \rho \in \Delta(\mathcal{S})$ に対して、次が成り立つ。

$$
\begin{aligned}
V^*(\rho) - V^{\pi}(\rho) &\leq \left\| \frac{d^{\pi^*}_{\rho}}{d^{\pi}_{\mu}} \right\|_{\infty} \max_{\bar{\pi}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu) \\
&\leq \frac{1}{1-\gamma} \left\| \frac{d^{\pi^*}_{\rho}}{\mu} \right\|_{\infty} \max_{\bar{\pi}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu)
\end{aligned}
$$

**補題4.1の証明**
$$
\begin{align*}
V^*(\rho) - V^{\pi}(\rho) &= \frac{1}{1-\gamma} \sum_{s, a} d^{\pi^*}_{\rho}(s) \pi^*(a|s) A^{\pi}(s, a) \\
&\leq \frac{1}{1-\gamma} \sum_{s, a} d^{\pi^*}_{\rho}(s) \max_{\bar{a}} A^{\pi}(s, \bar{a}) \\
&= \frac{1}{1-\gamma} \sum_{s} \frac{d^{\pi^*}_{\rho}(s)}{d^{\pi}_{\mu}(s)} \cdot d^{\pi}_{\mu}(s) \max_{\bar{a}} A^{\pi}(s, \bar{a}) \\
&\leq \frac{1}{1-\gamma} \left( \max_s \frac{d^{\pi^*}_{\rho}(s)}{d^{\pi}_{\mu}(s)} \right) \sum_{s} d^{\pi}_{\mu}(s) \max_{\bar{a}} A^{\pi}(s, \bar{a}) \tag{8}
\end{align*}
$$
次に、右辺の$\sigma$以降について考えると、

$$
\begin{aligned}
\sum_s \frac{d^{\pi}_{\mu}(s)}{1-\gamma} \max_{\bar{a}} A^{\pi}(s, \bar{a}) &= \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \sum_{s, a} \frac{d^{\pi}_{\mu}(s)}{1-\gamma} \bar{\pi}(a|s) A^{\pi}(s, a) \\
&= \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \sum_{s, a} \frac{d^{\pi}_{\mu}(s)}{1-\gamma} (\bar{\pi}(a|s) - \pi(a|s)) A^{\pi}(s, a) \\
&= \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \sum_{s, a} \frac{d^{\pi}_{\mu}(s)}{1-\gamma} (\bar{\pi}(a|s) - \pi(a|s)) Q^{\pi}(s, a) \\
&= \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu)
\end{aligned}
$$
よって、以下のようになります。
$$
\begin{aligned}
V^*(\rho) - V^{\pi}(\rho) &\leq \left\| \frac{d^{\pi^*}_{\rho}}{d^{\pi}_{\mu}} \right\|_{\infty} \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu) \\
&\leq \frac{1}{1-\gamma} \left\| \frac{d^{\pi^*}_{\rho}}{\mu} \right\|_{\infty} \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu)
\end{aligned}
$$

勾配支配が直接パラメータ化において成立することが分かりました。では、勾配を用いて最適方策を求めるアルゴリズムの収束について考えましょう。具体的にはSample Complexityをアルゴリズム(射影勾配上昇法)について求めましょう。射影勾配上昇は、以下のように方策を更新します。
$$
\begin{equation*}
\pi^{(t+1)} = P_{\Delta(\mathcal{A})^{|\mathcal{S}|}} \left( \pi^{(t)} + \eta \nabla_{\pi} V^{(t)}(\mu) \right) \tag{9}
\end{equation*}
$$
$P_{\Delta(\mathcal{A})^{|\mathcal{S}|}}$は、単体$\Delta(\mathcal{A})^{|\mathcal{S}|}$への射影を意味します。

次に、射影勾配法のSample Complexityを示しましょう。

**定理4.1.**

ステップサイズ $\eta = \frac{(1-\gamma)^3}{2\gamma|\mathcal{A}|}$ を持つ、$V^{\pi}(\mu)$ 上の射影勾配上昇アルゴリズム（9）は、すべての分布 $\rho \in \Delta(\mathcal{S})$ について、次を満たす。

$$
\min_{t<T} \{ V^*(\rho) - V^{(t)}(\rho) \} \leq \epsilon \quad \text{ whenever } \quad T > \frac{64\gamma|\mathcal{S}||\mathcal{A}|}{(1-\gamma)^6 \epsilon^2} \left\| \frac{d^{\pi^*}_{\rho}}{\mu} \right\|_{\infty}^2
$$

**定理4.1の証明**
証明のためにの定義、仮定、命題、補題、定理を示します。

**定義B.1（一次停留性)**
方策$\pi_{\theta} \in \Delta(\mathcal{A})^{|S|}$は、次の場合に初期状態分布$\mu$に関して$\epsilon$停留である。

$$
G(\pi_{\theta}) := \max_{\pi_{\theta}+\delta \in \Delta(\mathcal{A})^{|S|}, \|\delta\|_2 \leq 1} \delta^{\top} \nabla_{\pi} V^{\pi_{\theta}}(\mu) \leq \epsilon
$$
価値関数の勾配がε以下の場合、停留しているといえます。

**命題B.1(勾配写像)**

$V^{\pi}(\mu)$は$\pi$において$\beta$-滑らかであるとする。勾配写像を次のように定義する。

$$
G^{\eta}(\pi) = \frac{1}{\eta} \left( P_{\Delta(\mathcal{A})^{|S|}} (\pi + \eta \nabla_{\pi} V^{\pi}(\mu)) - \pi \right),
$$

方策から勾配を写像する関数を定義しています。

**補題D.3（直接パラメータ化の滑らかさ）**

すべての開始状態$s_0$について、

$$
\| \nabla_\pi V^\pi(s_0) - \nabla_\pi V^{\pi'}(s_0) \|_2 \leq \frac{2\gamma |\mathcal{A}|}{(1-\gamma)^3} \| \pi - \pi' \|_2
$$

が成り立つ。

**仮定E.1.**
 $f : \mathbb{R}^d \rightarrow (-\infty, \infty)$ は適切かつ閉であり、$\operatorname{dom}(f)$は凸であり、$f$ は $\operatorname{int}(\operatorname{dom}(f))$上で$\beta$滑らかであるとする。

**定理E.1 ([Beck, 2017] の定理10.15)**

仮定E.1が成り立ち、$\min_{x \in C} \{ f(x) \}$($C$ は非空の閉凸集合)を解くために勾配降下アルゴリズムで生成されたシーケンス $\{x_k\}_{k \geq 0}$ がステップサイズ $\eta=1/\beta$ を持つとする。すると、

1.  シーケンス $\{ F(x_t) \}_{t \geq 0}$ は非増加である。
2.  $t \rightarrow \infty$ につれて、$G^{\eta}(x_t) \rightarrow 0$ となる。
3.  $\min_{t=0, 1, \dots, T-1} \| G^\eta(x_t) \| \leq \frac{\sqrt{2 \beta (f(x_0) - f(x^*))}}{\sqrt{T}}$ となる。



##### 証明
補題D.3より、状態価値関数は全ての状態に対して$\beta$-滑らかとなります。これにより、仮定E.1が満たされているので定理E.1が成立します。3番目の項目より、ステップサイズ$\eta = \frac{1}{\beta}$の$G^{\eta}(\pi)$について、次が得られます。

$$
\min_{t=0,1,\dots,T-1} \| G^{\eta}(\pi^{(t)}) \|_2 \leq \frac{\sqrt{2\beta(V^*(\mu)-V^{(0)}(\mu))}}{\sqrt{T}}
$$

次に、命題B.1を利用して以下のように変形します。

$$
\min_{t=0,1,\dots,T} \max_{\pi^{(t)} + \delta \in \Delta(\mathcal{A})^{|\mathcal{S}|}, \|\delta\|_2 \leq 1} \delta^{\top} \nabla_{\pi} V^{\pi^{(t+1)}}(\mu) \leq (\eta \beta + 1) \frac{\sqrt{2\beta(V^*(\mu) - V^{(0)}(\mu))}}{\sqrt{T}}
$$
さらに、
$$
\begin{aligned}
\max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu) &= 2 \sqrt{|\mathcal{S}|} \max_{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \frac{1}{2 \sqrt{|\mathcal{S}|}} (\bar{\pi} - \pi)^{\top} \nabla_{\pi} V^{\pi}(\mu) \\
& \leq 2 \sqrt{|\mathcal{S}|} \max_{\pi + \delta \in \Delta(\mathcal{A})^{|\mathcal{S}|}, \|\delta\|_2 \leq 1} \delta^{\top} \nabla_{\pi} V^{\pi}(\mu)
\end{aligned}
$$

補題4.1および$\eta \beta = 1$を使用すると、

$$
\min_{t=0,1,\dots,T} V^*(\rho) - V^{(t)}(\rho) \leq \frac{4\sqrt{|\mathcal{S}|}}{1-\gamma} \left\| \frac{d^{\pi^*}_\rho}{\mu} \right\|_\infty \frac{\sqrt{2\beta (V^*(\mu) - V^{(0)}(\mu))}}{\sqrt{T}}
$$
となります。最後に、右辺について$\epsilon$の上界を作ると
$$
\frac{4\sqrt{|\mathcal{S}|}}{1-\gamma} \left\| \frac{d^{\pi^*}_\rho}{\mu} \right\|_\infty \frac{\sqrt{2\beta (V^*(\mu) - V^{(0)}(\mu))}}{\sqrt{T}} \leq \epsilon
$$

Tを変形させてSample complexityを導くことができます。
$$
T \geq \frac{32 |\mathcal{S}| \beta(V^*(\mu) - V^{(0)}(\mu))}{(1-\gamma)^2 \epsilon^2} \left\| \frac{d^{\pi^*}_\rho}{\mu} \right\|^2_\infty
$$